In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
import json
import os
import subprocess
import wget

my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")
my_options.add_argument("--start-maximized")
my_options.add_argument("--incognito")
my_options.add_argument("--disable-popup-blocking")
my_options.add_argument("--disable-notifications")
my_options.add_argument("--lang=zh-TW")
driver = webdriver.Chrome(
    options = my_options
)

folderPath = 'youtube'
if not os.path.exists(folderPath):
    os.mkdir(folderPath)

listData = []

In [28]:
def visit():
    driver.get('https://www.youtube.com/')

def search():
    txtInput = driver.find_element(
        By.CSS_SELECTOR, "input#search"
    )
    txtInput.send_keys('烏薩奇')
    
    sleep(1)

    txtInput.submit()

    sleep(1)

def filterFunc():
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR,'ytd-button-renderer.style-scope.ytd-search-header-renderer')
            )
        )

        driver.find_element(
            By.CSS_SELECTOR,'ytd-button-renderer.style-scope.ytd-search-header-renderer'
        ).click()

        sleep(2)

        driver.find_elements(
            By.CSS_SELECTOR,'div#options a#endpoint.yt-simple-endpoint.style-scope.ytd-search-filter-renderer'
        )[9].click()

        sleep(2)

    except TimeoutException:
        print('等待逾時')

def scroll():
    innerHeight = 0
    offset = 0
    count = 0
    limit = 3

    while count < limit:
        offset = driver.execute_script(
            'return document.documentElement.scrollHeight;'
        )

        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset},
                behavior: 'smooth'
            }})
        ''')

        break 
        # 只捲動一次
        # sleep(3)

        # innerHeight = driver.execute_script(
        #     'return document.documentElement.scrollHeight;'
        # )

        # if offset == innerHeight:
        #     count += 1

def parse():
    global listData
    listData.clear()

    elements = driver.find_elements(
        By.CSS_SELECTOR, 
        'ytd-video-renderer.style-scope.ytd-item-section-renderer'
    )

    for elm in elements:
        print("=" * 30)

        img = elm.find_element(By.CSS_SELECTOR, "a#thumbnail img")
        imgSrc = img.get_attribute('src')
        print(imgSrc)

        a = elm.find_element(By.CSS_SELECTOR, "a#video-title")
        aTitle = a.get_attribute('innerText')
        print(aTitle)

        aLink = a.get_attribute('href')
        print(aLink)

        strDelimiter = ''
        if 'shorts' in aLink:
            strDelimiter = '/shorts/'
        else:
            strDelimiter = 'v='
        youtube_id = aLink.split(strDelimiter)[1]
        youtube_id = youtube_id.split('&pp')[0]
        print(youtube_id)

        listData.append({
            "id": youtube_id,
            "title": aTitle,
            "link": aLink,
            "img": imgSrc
        })

def saveJson():
    with open(f'{folderPath}/youtube.json', 'w', encoding='utf-8') as file:
        file.write(
            json.dumps(
                listData,
                ensure_ascii=False,
                indent=4
            )
        )

def close():
    driver.quit()

In [ ]:
visit()
search()
filterFunc()
scroll()
parse()
saveJson()

In [ ]:
close()